In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np

In [2]:
TOP_UNIV_BASE = "https://www.topuniversities.com"
TOP_UNIV_WEBSITE = "https://www.topuniversities.com/university-rankings/world-university-rankings/2018"
# This is the file containing the general informations about university in the topuniversities website.
TOP_UNIV_DATA_INDICATOR = "https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051_indicators.txt"

In [6]:
# We get the file containing information about the universities
data_indicator = requests.get(TOP_UNIV_DATA_INDICATOR)

indicator_data = json.loads(data_indicator.text)

In [7]:
# Utility function to parse integers
def parse_int(string):
    tmp = string.replace(',', '')
    return int(tmp)

# For each university entry in the indicator file, we get the detail of the university.
# This method returns a list with every information requested
def get_university_info_from_indicator(raw_university):
    overview_soup = BeautifulSoup(raw_university["uni"], 'html.parser')    
    
    name = overview_soup.a.text
    rank = raw_university["overall_rank"]
    region = raw_university["region"]
    country = raw_university["location"]
    
    # Append
    details_request = requests.get(TOP_UNIV_BASE + "/" +  overview_soup.a.attrs['href'])
    details_soup = BeautifulSoup(details_request.text, 'html.parser')
    
    try:      
        total_staff = parse_int(details_soup.find("div", {"class": "total faculty"}).find("div", {"class" : "number"}).text)
        inter_staff = parse_int(details_soup.find("div", {"class": "inter faculty"}).find("div", {"class" : "number"}).text)
        total_students = parse_int(details_soup.find("div", {"class": "total student"}).find("div", {"class": "number"}).text)
        inter_students = parse_int(details_soup.find("div", {"class": "total inter"}).find("div", {"class": "number"}).text)
    except:
        total_staff = np.nan
        inter_staff = np.nan
        total_students = np.nan
        inter_students = np.nan
        
    return [name, rank, country, region, inter_staff, total_staff, inter_students, total_students]


In [8]:
# The main loop, for each university in the indicator file.
univ_list = []

for i, raw_university in enumerate(indicator_data["data"][0:200]):
    print(i, end="\r")
    univ_list.append(get_university_info_from_indicator(raw_university))
    
        
top_univ_df = pd.DataFrame(univ_list, columns=["name", "rank", "country", "region", "inter_staff", "total_staff", "inter_students", "total_students"])

In [9]:
top_univ_df

,name,rank,country,region,inter_staff,total_staff,inter_students,total_students
0,Massachusetts Institute of Technology (MIT),1,United States,North America,1679.0,2982.0,3717.0,11067.0
1,Stanford University,2,United States,North America,2042.0,4285.0,3611.0,15878.0
2,Harvard University,3,United States,North America,1311.0,4350.0,5266.0,22429.0
3,California Institute of Technology (Caltech),4,United States,North America,350.0,953.0,647.0,2255.0
4,University of Cambridge,5,United Kingdom,Europe,2278.0,5490.0,6699.0,18770.0
5,University of Oxford,6,United Kingdom,Europe,2964.0,6750.0,7353.0,19720.0
6,UCL (University College London),7,United Kingdom,Europe,2554.0,6345.0,14854.0,31080.0
7,Imperial College London,8,United Kingdom,Europe,2071.0,3930.0,8746.0,16090.0
8,University of Chicago,9,United States,North America,635.0,2449.0,3379.0,13557.0
9,ETH Zurich - Swiss Federal Institute of Techno...,10,Switzerland,Europe,1886.0,2477.0,7563.0,19815.0


In [10]:
top_univ_df.to_csv("top_univ_df.csv")

In [3]:
top_univ_df = pd.read_csv("top_univ_df.csv", index_col=False, usecols=range(1, 9))

In [5]:
def print_best_university_with_ratio(df): 
    sorted_df = df.sort_values("ratio", ascending=False)
    display(sorted_df.head(1))
    
def print_best_university_with_ratio_grouped(df_grouped):
    display(df_grouped.apply(lambda x: x[x["ratio"] == x["ratio"].max()]).reset_index(1).drop("level_1", 1))

def append_column_to_dataframe(dataframe, column, column_name):
    output_df = dataframe.copy()
    output_df.loc[:, column_name] = column
    return output_df

members_to_studends_ratio = top_univ_df.total_staff / (top_univ_df.total_students + top_univ_df.total_staff)
top_univ_df_with_ratio_a = append_column_to_dataframe(top_univ_df, members_to_studends_ratio, "ratio")

inter_students_ratio = top_univ_df.inter_students / top_univ_df.total_students
top_univ_df_with_ratio_b = append_column_to_dataframe(top_univ_df, inter_students_ratio, "ratio")


print("The best university in term of ratio between faculty members and students is")
print_best_university_with_ratio(top_univ_df_with_ratio_a)
print("\n\nThe best university in term of  ratio of international students is")
print_best_university_with_ratio(top_univ_df_with_ratio_b)

print("\n\nThe best university in term of ratio between faculty members and students per country are")
print_best_university_with_ratio_grouped(top_univ_df_with_ratio_a.groupby("country"))
print("\n\nThe best university in term of  ratio of international students per country are")
print_best_university_with_ratio_grouped(top_univ_df_with_ratio_b.groupby("country"))

print("\n\nThe best university in term of ratio between faculty members and students per region are")
print_best_university_with_ratio_grouped(top_univ_df_with_ratio_a.groupby("region"))
print("\n\nThe best university in term of  ratio of international students per region are")
print_best_university_with_ratio_grouped(top_univ_df_with_ratio_b.groupby("region"))

The best university in term of ratio between faculty members and students is


,name,rank,country,region,inter_staff,total_staff,inter_students,total_students,ratio
3,California Institute of Technology (Caltech),4,United States,North America,350.0,953.0,647.0,2255.0,0.29707




The best university in term of  ratio of international students is


,name,rank,country,region,inter_staff,total_staff,inter_students,total_students,ratio
34,London School of Economics and Political Scien...,35,United Kingdom,Europe,687.0,1088.0,6748.0,9760.0,0.691393




The best university in term of ratio between faculty members and students per country are


...

,name,rank,country,region,inter_staff,total_staff,inter_students,total_students,ratio
country,,,,,,,,,
Argentina,Universidad de Buenos Aires (UBA),75,Argentina,Latin America,3165.0,16421.0,27109.0,122301.0,0.221658
Australia,The University of Melbourne,41,Australia,Oceania,1477.0,3311.0,18030.0,42182.0,0.427434
Austria,University of Vienna,154,Austria,Europe,1400.0,3411.0,14468.0,45967.0,0.314748
Belgium,Vrije Universiteit Brussel (VUB),182,Belgium,Europe,515.0,1792.0,1853.0,9284.0,0.199591
Brazil,Universidade Estadual de Campinas (Unicamp),182,Brazil,Latin America,109.0,1968.0,966.0,26572.0,0.036354
Canada,McGill University,32,Canada,North America,1220.0,3646.0,9540.0,28837.0,0.330825
Chile,Universidad de Chile,201,Chile,Latin America,127.0,2256.0,2134.0,38848.0,0.054932
China,Peking University,38,China,Asia,1038.0,5185.0,7090.0,42136.0,0.168265
Denmark,Technical University of Denmark,116,Denmark,Europe,966.0,2117.0,2098.0,8878.0,0.236314




The best university in term of ratio between faculty members and students per region are


,name,rank,country,region,inter_staff,total_staff,inter_students,total_students,ratio
region,,,,,,,,,
Africa,University of Cape Town,191,South Africa,Africa,379.0,1733.0,3325.0,19593.0,0.081262
Asia,Pohang University of Science And Technology (P...,71,South Korea,Asia,113.0,664.0,126.0,3117.0,0.175615
Europe,University of Oxford,6,United Kingdom,Europe,2964.0,6750.0,7353.0,19720.0,0.255006
Latin America,Instituto Tecnológico y de Estudios Superiores...,199,Mexico,Latin America,821.0,1822.0,1412.0,13376.0,0.119884
North America,California Institute of Technology (Caltech),4,United States,North America,350.0,953.0,647.0,2255.0,0.297070
Oceania,The Australian National University,20,Australia,Oceania,927.0,1600.0,5551.0,14442.0,0.099738




The best university in term of  ratio of international students per region are


,name,rank,country,region,inter_staff,total_staff,inter_students,total_students,ratio
region,,,,,,,,,
Africa,University of Cape Town,191,South Africa,Africa,379.0,1733.0,3325.0,19593.0,0.169703
Asia,The University of Hong Kong,26,Hong Kong,Asia,2085.0,3012.0,8230.0,20214.0,0.407144
Europe,London School of Economics and Political Scien...,35,United Kingdom,Europe,687.0,1088.0,6748.0,9760.0,0.691393
Latin America,Universidad de Buenos Aires (UBA),75,Argentina,Latin America,3165.0,16421.0,27109.0,122301.0,0.221658
North America,Carnegie Mellon University,47,United States,North America,425.0,1342.0,6385.0,13356.0,0.478062
Oceania,The University of Melbourne,41,Australia,Oceania,1477.0,3311.0,18030.0,42182.0,0.427434
